In [32]:
import pandas as pd
import numpy as np
 
# Define the datasets and ablation types
datasets = ['adult', 'taxi', 'wine']
pipeline = ['Inc', 'Taxi', 'Wine']
ablation_types = ['full', 'no_variance_filter', 'no_dim_reduction', 'no_iterative_filter']

# Create mapping from dataset to pipeline name
dataset_to_pipeline = {dataset: pipe for dataset, pipe in zip(datasets, pipeline)}

# Create mapping for ablation names
ablation_name_map = {
    'full': 'FULL',
    'no_dim_reduction': 'NDR',
    'no_iterative_filter': 'NIS',
    'no_variance_filter': 'NVF'
}
 
# Create results dictionary
results = {}
 
base_path = "./results"
 
for dataset in datasets:
    print(dataset)
    for ablation in ablation_types:
        print(ablation)
 
        key = f"{dataset.capitalize()}_{ablation}"
        
        try:
            # Read Silhouette and DBI
            cluster_quality_path = f"{base_path}/{dataset}/{ablation}/cluster_quality/overall_cluster_quality_scores.csv"
            cluster_df = pd.read_csv(cluster_quality_path)
            silhouette_avg = cluster_df['Silhouette_Score'].values[0]
            dbi_avg = cluster_df['Davies_Bouldin_Index'].values[0]
            
            # Read min and max silhouette from per_cluster_silhouette_metrics.csv
            per_cluster_path = f"{base_path}/{dataset}/{ablation}/cluster_quality/per_cluster_silhouette_metrics.csv"
            try:
                per_cluster_df = pd.read_csv(per_cluster_path)
                n_clusters = len(per_cluster_df)
                silhouette_min = per_cluster_df['silhouette_min'].min()
                silhouette_max = per_cluster_df['silhouette_max'].max()
            except:
                n_clusters = None
                silhouette_min = None
                silhouette_max = None
            
            # Read representative quality detailed for CV metrics
            rep_quality_path = f"{base_path}/{dataset}/{ablation}/representative_quality/representative_quality_detailed.csv"
            try:
                rep_df = pd.read_csv(rep_quality_path)
                
                # Average these counts across clusters
                selected_cvs = rep_df[rep_df['Type'] == 'Selected'].groupby('Cluster')['CV'].sum()
                non_selected_cvs = rep_df[rep_df['Type'] == 'Non-Selected'].groupby('Cluster')['CV'].sum()

                # Metrics: average number of selected metrics per cluster
                metrics_avg =  rep_df[rep_df['Type'] == 'Selected'].groupby('Cluster').count()["Metric"].mean()

                # Get CV statistics for selected
                selected_cv_avg = selected_cvs.mean()
                selected_cv_min = selected_cvs.min()
                selected_cv_max = selected_cvs.max()
                
                # Get CV statistics for non-selected
                non_selected_cv_avg = non_selected_cvs.mean()
                non_selected_cv_min = non_selected_cvs.min()
                non_selected_cv_max = non_selected_cvs.max()
                
                # Compute ratio of selected vs non-selected CV
                ratio_avg = selected_cv_avg / non_selected_cv_avg if non_selected_cv_avg != 0 else None
                ratio_min = selected_cv_min / non_selected_cv_min if non_selected_cv_min != 0 else None
                ratio_max = selected_cv_max / non_selected_cv_max if non_selected_cv_max != 0 else None
                
                # Handle NaN/Inf cases
                if ratio_avg is not None and (pd.isna(ratio_avg) or np.isinf(ratio_avg)):
                    ratio_avg = None
                if ratio_min is not None and (pd.isna(ratio_min) or np.isinf(ratio_min)):
                    ratio_min = None
                if ratio_max is not None and (pd.isna(ratio_max) or np.isinf(ratio_max)):
                    ratio_max = None
                    
                # Handle metrics_avg NaN
                try:
                    if metrics_avg != metrics_avg:  # NaN check
                        metrics_avg = None
                except:
                    metrics_avg = None

            except Exception as e:
                print(f"Could not load representative quality for {key}: {e}")
                ratio_avg = None
                ratio_min = None
                ratio_max = None
                metrics_avg = None
            
            # Read QSE - try multiple possible filenames
            qse_path = None
            for filename in ['qse_metrics.csv', 'qse_best_per_cluster.csv', 'qse_all_rules.csv']:
                test_path = f"{base_path}/{dataset}/{ablation}/explanation_quality/{filename}"
                try:
                    qse_df = pd.read_csv(test_path)
                    qse_path = test_path
                    break
                except:
                    continue
            
            if qse_path:
                qse_df = pd.read_csv(qse_path)
                if 'qse' in qse_df.columns:
                    qse_avg = qse_df['qse'].mean()
                elif 'QSE' in qse_df.columns:
                    qse_avg = qse_df['QSE'].mean()
                else:
                    # Try the first numeric column
                    numeric_cols = qse_df.select_dtypes(include=[np.number]).columns
                    qse_avg = qse_df[numeric_cols[0]].mean() if len(numeric_cols) > 0 else None
            else:
                qse_avg = None
            
            # Read F1 Score from predictive quality metrics
            f1_path = f"{base_path}/{dataset}/{ablation}/predictive_quality/predictive_quality_metrics.csv"
            try:
                f1_df = pd.read_csv(f1_path)
                if 'F1 Score' in f1_df.columns:
                    f1_scores = f1_df['F1 Score'].values
                    f1_avg = f1_scores.mean()
                    # Check if all zeros
                    if np.all(f1_scores == 0):
                        f1_str = "0"
                    else:
                        f1_str = f"{f1_avg:.4f}"
                        f1_str = "0"

            except Exception as e:
                print(f"Could not load F1 scores for {key}: {e}")
                f1_str = "0"
            
            results[key] = {
                'Pipeline': dataset_to_pipeline[dataset],
                'Ablation': ablation_name_map.get(ablation, ablation.replace('_', ' ').title()),
                'N_Clusters': n_clusters if n_clusters is not None else "N/A",
                'Metrics_avg': f"{int(metrics_avg)}" if metrics_avg is not None else "N/A",
                'SiS_avg': f"{silhouette_avg:.4f}",
                'SiS_min': f"{silhouette_min:.4f}" if silhouette_min is not None else "N/A",
                'SiS_max': f"{silhouette_max:.4f}" if silhouette_max is not None else "N/A",
                'DBI_avg': f"{dbi_avg:.4f}",
                'Ratio_avg': f"{ratio_avg:.4f}" if ratio_avg is not None else "N/A",
                'Ratio_min': f"{ratio_min:.4f}" if ratio_min is not None else "N/A",
                'Ratio_max': f"{ratio_max:.4f}" if ratio_max is not None else "N/A",
                'F1_avg': f1_str,
                'QSE_avg': f"{qse_avg:.4f}" if qse_avg is not None else "N/A"
            }
        except Exception as e:
            print(f"Could not load {key}: {e}")
 
# Create dataframe from results and sort by Pipeline then Ablation
results_df = pd.DataFrame(list(results.values()))
results_df = results_df.sort_values(by=['Pipeline', 'Ablation']).reset_index(drop=True)

# Display the table
print("\nResults Table for Paper (sorted by Pipeline):")
print("=" * 160)
print(results_df.to_string(index=False))
print("=" * 160)

# Create a version with bolded winners for CSV
csv_df = results_df.copy()

# Define which columns are "higher is better" and "lower is better"
higher_is_better = ['SiS_avg', 'SiS_min', 'SiS_max',  'F1_avg', 'QSE_avg']  # These want max
lower_is_better = ['DBI_avg', 'Ratio_avg', 'Ratio_min', 'Ratio_max']  # These want min

# Bold the winners in each column, per dataset
for col in csv_df.columns:
    if col in ['Pipeline', 'Ablation', 'N_Clusters', 'Metrics_avg']:
        continue  # Skip non-numeric or non-comparative columns
    
    try:
        # For each pipeline, bold the best value
        for pipeline_name in csv_df['Pipeline'].unique():
            pipeline_indices = csv_df[csv_df['Pipeline'] == pipeline_name].index
            
            # Convert strings to floats for comparison within this pipeline
            numeric_values = pd.to_numeric(csv_df.loc[pipeline_indices, col], errors='coerce')
            
            if numeric_values.isna().all():
                continue  # Skip if all N/A
            
            if col in higher_is_better:
                # Bold the maximum value(s) in this pipeline
                max_val = numeric_values.max()
                if pd.notna(max_val):
                    idx = numeric_values[numeric_values == max_val].index[0]
                    current_val = csv_df.loc[idx, col]
                    if not current_val.startswith('\\textbf'):
                        csv_df.loc[idx, col] = f"\\textbf{{{current_val}}}"
            
            elif col in lower_is_better:
                # Bold the minimum value(s) in this pipeline
                min_val = numeric_values.min()
                if pd.notna(min_val):
                    idx = numeric_values[numeric_values == min_val].index[0]
                    current_val = csv_df.loc[idx, col]
                    if not current_val.startswith('\\textbf'):
                        csv_df.loc[idx, col] = f"\\textbf{{{current_val}}}"
    except:
        continue

# Color the "Full" ablation rows in blue (entire row)
full_indices = csv_df[csv_df['Ablation'] == 'FULL'].index
for idx in full_indices:
    for col in csv_df.columns:
        try:
            current_val = csv_df.loc[idx, col]
            # Convert to string to safely call startswith
            current_val_str = str(current_val)
            if not current_val_str.startswith('\\blue'):
                # If already bolded, wrap the entire textbf with blue, otherwise just wrap value
                if current_val_str.startswith('\\textbf'):
                    csv_df.loc[idx, col] = f"\\blue{{{current_val_str}}}"
                else:
                    csv_df.loc[idx, col] = f"\\blue{{{current_val_str}}}"
        except:
            continue

# Save to CSV
csv_df.to_csv("./results_table_for_paper.csv", index=False)
print("\nTable saved to: ./results_table_for_paper.csv")

adult
full
no_variance_filter
Could not load F1 scores for Adult_no_variance_filter: [Errno 2] No such file or directory: './results/adult/no_variance_filter/predictive_quality/predictive_quality_metrics.csv'
no_dim_reduction
Could not load F1 scores for Adult_no_dim_reduction: [Errno 2] No such file or directory: './results/adult/no_dim_reduction/predictive_quality/predictive_quality_metrics.csv'
no_iterative_filter
taxi
full
no_variance_filter
Could not load F1 scores for Taxi_no_variance_filter: [Errno 2] No such file or directory: './results/taxi/no_variance_filter/predictive_quality/predictive_quality_metrics.csv'
no_dim_reduction
no_iterative_filter
wine
full
no_variance_filter
no_dim_reduction
no_iterative_filter

Results Table for Paper (sorted by Pipeline):
Pipeline Ablation  N_Clusters Metrics_avg SiS_avg SiS_min SiS_max DBI_avg Ratio_avg Ratio_min Ratio_max F1_avg QSE_avg
     Inc     FULL           4           5  0.9216  0.8153  0.9656  0.1395    0.0271    0.0273    0.0153 

/var/folders/2v/qdpty4297ln507zdcjn9040h0000gn/T/ipykernel_95012/2325361582.py:226: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '\blue{4}' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  csv_df.loc[idx, col] = f"\\blue{{{current_val_str}}}"
